<a href="https://colab.research.google.com/github/LuFernandez/twin-peaks-generator/blob/master/bi_lstm_refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twin Peaks Scripts Generation using Bidirectional LSTM
## Lucero Fernandez - 2021

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Imports

In [2]:
from __future__ import print_function
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Flatten, Bidirectional, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback
from keras.metrics import categorical_accuracy
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle
import json

In [3]:
#chequeo estar usando la gpu y me fijo cuál
import tensorflow as tf 
print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5828747287148950582
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6553748542975363629
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


Tokenización -> paquete spacy 

In [4]:
#import spacy, and english model
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


# paths

In [5]:
from datetime import datetime
import pytz

tz_bsas = pytz.timezone('America/Buenos_Aires') 
datetime_bsas = datetime.now(tz_bsas)
date = datetime_bsas.strftime("%Y-%m-%d-%Hh_%Mm")

In [6]:
data_dir = '/content/drive/MyDrive/twin-peaks-generator/data'# data directory containing input.txt
save_dir = '/content/drive/MyDrive/twin-peaks-generator/save/' + date # directory to store models
os.mkdir(save_dir)
seq_length = 35 # sequence length
sequences_step = 1 #step to create sequences

In [7]:
# file_list = ["100","101","102","103","104","105","201","202","203","204","205","206","207","208","209","210","211","212","213","214","215","216","217","218","219","220","221","222"]
file_list = ["101","202","210","216"]
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

# Leer la data

Defino función para crear lista de palabras a partir del texto y luego procesamos esta lista con la función nlp de spacy, lo que devuelve todas las palabras en minúscula. Luego quitamos los \n.

In [8]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n", '\n\n\n', '\n\n\n\n', '\n\n\n\n\n','\n\n\n\n\n\n','\n\n\n\n\n\n\n','\n\n\n\n\n\n\n\n', '\n\n\n\n\n\n\n\n\n',
                             '\n\n\n\n\n\n\n\n\n\n', '\n\n\n\n\n\n\n\n\n\n\n', '\n\n\n\n\n\n\n\n\n \n', '\n\n\n\n\n\n\n\n ', '\n\n\n\n\n\n\n\n \n', '\n\n\n\n\n\n\n ',
                             '\n\n\n\n\n\n\n \n', '\n\n\n\n\n\n\n \n\n\n\n', '\n\n\n\n\n\n ', '\n\n\n\n\n\n \n', '\n\n\n\n\n\n \n\n\n', '\n\n\n\n\n\n \n\n\n\n\n',
                             '\n\n\n\n\n\n  ', '\n\n\n\n\n ', '\n\n\n\n\n \n', '\n\n\n\n\n \n\n', '\n\n\n\n\n \n\n\n', '\n\n\n\n\n \n\n\n\n\n', '\n\n\n\n\n  ',
                             '\n\n\n\n ', '\n\n\n\n \n', '\n\n\n\n \n\n', '\n\n\n\n \n\n\n\n', '\n\n\n ', '\n\n ', '\n\n \n', '\n\n \n\n\n\n', '\n ', '\n \n',
                              '\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

Creamos la lista de palabras para un total de 42004 palabras.

In [9]:
wordlist = []
for file_name in file_list:
    input_file = os.path.join(data_dir, file_name + ".txt")
    #read data
    with codecs.open(input_file, "r") as f:
        data = f.read()
    #create sentences
    doc = nlp(data)
    wl = create_wordlist(doc)
    wordlist = wordlist + wl

In [10]:
print(len(wordlist))
print(wordlist)

42004
['act', 'one', 'fade', 'in', ':', 'ext', '.', 'great', 'northern', 'hotel', '-', 'day', 'dawn', 'breaks', 'over', 'the', 'great', 'northern', '.', 'cut', 'to', ':', 'int', '.', 'great', 'northern', 'hotel', 'room', '-', 'day', 'we', 'hear', 'him', 'before', 'we', 'see', 'him', ',', 'but', 'dale', 'cooper', 'is', 'perched', 'six', 'inches', 'above', 'the', 'floor', 'in', 'a', 'one', '-', 'handed', 'yoga', '"', 'frog', '"', 'position', ',', 'wearing', 'boxer', 'shorts', 'and', 'a', 'pair', 'of', 'socks', ',', 'talking', 'into', 'the', 'tape', 'recorder', 'which', 'is', 'sitting', 'on', 'the', 'carpet', 'near', 'his', 'head', '.', 'cooper', 'diane', '...', '6:18', 'a.m.', ',', 'room', '315', ',', 'great', 'northern', 'hotel', 'up', 'here', 'in', 'twin', 'peaks', '.', 'slept', 'pretty', 'well', '.', 'nonsmoking', 'room', '.', 'no', 'tobacco', 'smell', '.', 'that', "'s", 'a', 'nice', 'consideration', 'for', 'the', 'business', 'traveler', '.', 'a', 'hint', 'of', 'douglas', 'fir', 'need

## Crear el diccionario

Creamos el diccionario. A cada palabra en el vocabulario le asignamos un índice. Se mapean las palabras más usadas, las más comunes con índices más chicos.

In [11]:
# contamos el numero de palabras
word_counts = collections.Counter(wordlist)

# Mapeamos de índice a palabra; el diccionario
words = [x[0] for x in word_counts.most_common()]
# vocabulary_inv = list(sorted(vocabulary_inv))

# mapeamos de palabra a índice
vocab = {x: i for i, x in enumerate(words)}
# words = [x[0] for x in word_counts.most_common()]

#tamaño del vocabulario
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
# with open(os.path.join(vocab_file), 'wb') as f:
#     cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  4609


In [12]:
print(vocab)

{'.': 0, ',': 1, 'the': 2, 'to': 3, 'a': 4, 'i': 5, 'you': 6, "'s": 7, 'and': 8, 'of': 9, '?': 10, 'cooper': 11, 'in': 12, ')': 13, '(': 14, '-': 15, 'it': 16, 'her': 17, 'she': 18, 'he': 19, ':': 20, 'on': 21, 'his': 22, 'that': 23, "n't": 24, 'at': 25, '...': 26, 'is': 27, 'was': 28, 'for': 29, 'with': 30, 'truman': 31, 'me': 32, 'do': 33, 'we': 34, 'audrey': 35, 'out': 36, 'my': 37, 'what': 38, 'this': 39, 'him': 40, 'as': 41, 'cut': 42, 'up': 43, 'bobby': 44, 'but': 45, 'norma': 46, 'james': 47, 'donna': 48, 'int': 49, 'have': 50, 'one': 51, 'be': 52, 'not': 53, 'know': 54, 'ben': 55, 'your': 56, 'shelly': 57, 'no': 58, "'m": 59, 'day': 60, 'about': 61, 'josie': 62, 'from': 63, 'leo': 64, 'like': 65, 'did': 66, 'are': 67, 'catherine': 68, 'room': 69, 'just': 70, 'all': 71, 'pete': 72, 'then': 73, 'night': 74, 'into': 75, 'down': 76, 'back': 77, 'there': 78, "'ve": 79, "'ll": 80, 'they': 81, 'so': 82, "'re": 83, 'laura': 84, 'hank': 85, '"': 86, 'here': 87, 'can': 88, 'looks': 89, '

## Creamos secuencias
Creamos la input data de la LSTM. Se crean dos listas:
 - **sequences**: esta lista contendrá las secuencias de palabras usadas para entrenar el modelo,
 - **next_words**: esta lista contendrá las próximas palabras para cada secuencia en la lista **sequences**.
 
Vamos a entrenar al modelo con secuencias de 35 palabras (seq_length = 35).
Entonces para crear la primera secuencia de palabras tomamos las primeras 35 palabras de la lista **wordlist**. La palabra 36 es la próxima palabra de esa lista y la agregamos a **next_words**.

Luego se avanza con un step de 1 (sequences_step = 1) en la lista de palabras para crear la siguiente secuencia de palabras y buscar la segunda "next word".

Iteramos hasta llegar al final de la lista de palabras.

In [13]:
#creamos secuencias
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 41969


Iterando sobre la lista, creamos 41969 secuencias de palabras y para cada una obtenemos la próxima palabra a predecir.

El siguiente paso a transformarlas en dígitos para ser el input de la LSTM.
La palabra no está representada simplemente por su índice en el vocabulario entonces no se puede mapear así. Ergo, reorganizamos la secuencia de palabras como matriz de bools.

Entonces creamos la matriz X e y:
 - X : con las siguientes dimensiones:
     - cantidad de secuencias (41969),
     - cantidad de palabras en las secuencias (35),
     - cantidad de palabras en el vocabulario (4609).
 - y : con las siguientes dimensiones:
     - cantidad de secuencias (41969),
     - cantidad de palabras en el vocabulario (4609).
 
Para cada palabra tomamos su índice en el vocabulario y colocamos un 1 en esa posición en la matriz.
La matriz X tendrá las secuencias de entrenamiento del modelo y la y las next_words.

In [14]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

print(np.shape(X))

(41969, 35, 4609)


# Construir el modelo

Para construir el modelo usamos Keras que nos permite una mayor abstracción al momento de diseñar la arquitectura.

La arquitectura de la red neuronal es la siguiente:
 - LSTM bidireccional,
 - con tamaño  y usando tanh como activation,
 - luego una capa de dropout de 0.6, lo que es bastante alto pero necesario para evitar divergencia.
 

La red nos da la probabilidad para cada palabra del vocabulario de que sea la próxima dada una oración. Por esto finalizamos con:

 - una capa simple densa del tamaño del vocabulario,
 - una capa de activación softmax.
 
Usamos Adam como optimizador y la loss la hacemos sobre la categorical crossentropy.

In [15]:
def bidirectional_lstm_model(seq_length, vocab_size, rnn_size, batch_size, learning_rate):
    print('Build LSTM model.')
    model = Sequential()
    #model.add(Bidirectional(LSTM(1024, activation="tanh", recurrent_activation = "sigmoid",return_sequences=True, recurrent_dropout = 0, use_bias=True,unroll= False),input_shape=(seq_length, vocab_size)))
    model.add(Bidirectional(LSTM(rnn_size, activation="tanh", recurrent_activation = "sigmoid", recurrent_dropout = 0, use_bias=True,unroll= False),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Lambda(lambda x: x / 0.5))
    model.add(Activation('softmax'))
    
    optimizer = Adam(learning_rate=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    return model

In [16]:
# #Hiper-parametros

# rnn_size = 256 # size of RNN
# batch_size = 32 # minibatch size
# seq_length = seq_length # sequence length
# num_epochs = 3 # number of epochs
# learning_rate = 0.001 #learning rate
# sequences_step = 1 #step to create sequences

In [17]:
# md = bidirectional_lstm_model(seq_length, vocab_size, rnn_size, batch_size, learning_rate)
# print(md.summary())

## Entrenar al modelo

Al momento de entrenar al modelo hacemos un shuffle sobre el dataset y extraemos un 10% para la validación.

In [18]:
def fit_model(num_epochs, batch):
  #fit the model
  callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
            ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences_lstm.{epoch:02d}-{val_loss:.2f}.hdf5',\
                            monitor='val_loss', verbose=0, mode='auto', period=2)]

  history = md.fit(X, y,
                  batch_size=batch,
                  shuffle=True,
                  epochs=num_epochs,
                  callbacks=[callbacks],
                  validation_split=0.1)

  #save the model
  md.save(save_dir + "/" + 'my_model_gen_sentences_lstm.final.hdf5')
  return history

# Generamos la frase

El modelo ahora predice la próxima palabra dada una secuencia de palabras.
Para generar texto se realiza lo siguiente:

 - se define una "seed", una secuencia de inicio de 35 palabras (35 siendo la cantidad de palabras necesarias para las secuencias),
 - le pedimos a la red que prediga la palabra 36,
 - actualizamos la secuencia shifteando las palabras un lugar (sequence_step = 1), agregando la palabra numero 36 al final y descartando la primera,
 - le pedimos a la red que prediga la palabra 37,
 - y se repite.
 
De esta manera generamos frases, palabra a palabra.

In [19]:
def load_vocabulary():
  #load vocabulary
  print("loading vocabulary...")
  vocab_file = os.path.join(save_dir, "words_vocab.pkl")

  with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
          words, vocab, vocabulary_inv = cPickle.load(f)

  vocab_size = len(words)
  return vocab_size

In [20]:
def load_the_model():
  # load the model
  print("loading model...")
  model = load_model(save_dir + "/" + 'my_model_gen_sentences_lstm.final.hdf5')
  return model

Para mejorar la generación de palabras y tunear la predicción introducimos una función especial para escoger las palabras.

No elegimos las palabras con mayor probabilidad sino que introducimos cierta incertidumbre para que sea menos aburrido el guión, y dejamos que la predicción elija palabras con menor probabilidad.

Este es el propósito de la función **sample**, randomizar un poco las probabilidades.

La probabilidad de una palabra de ser seleccionada depende directamente de su probabilidad de ser la próxima palabra. Para tunear esta probabilidad se introduce el parámetro de 'temperature' para suavizar o aumentar su valor.

In [21]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [22]:
#initiate sentences
def initiate_sentences(sentence, generated):
  seed_sentences = 'but the true test of any hotel , as you know , is that morning cup of coffee .'
  # generated = ''
  # sentence = []
  for i in range (seq_length):
      sentence.append("a")

  seed = seed_sentences.split()

  for i in range(len(seed)):
      sentence[seq_length-i-1]=seed[len(seed)-i-1]

  generated += ' '.join(sentence)
  #print('Generating text with the following seed: "' + ' '.join(sentence) + '"')
  return sentence, generated

In [23]:
def generate_text(sentence, generated):
  words_number = 500
  #generate the text
  for i in range(words_number):
      #create the vector
      x = np.zeros((1, seq_length, vocab_size))
      for t, word in enumerate(sentence):
          x[0, t, vocab[word]] = 1.
      #print(x.shape)

      #calculate next word
      preds = model.predict(x, verbose=0)[0]
      # next_index = sample(preds, 1.0)
      next_word = words[np.argmax(preds)]

      #add the next word to the text
      generated += " " + next_word
      # shift the sentence by one, and and the next word at its end
      sentence = sentence[1:] + [next_word]

  #print(generated)
  return sentence, generated


In [24]:
def post_process(generated):
  generated = generated.replace(" .", ".\n")
  generated = generated.replace(" )", ")")
  generated = generated.replace("( ", "(")
  generated = generated.replace(" '", "'")
  generated = generated.replace(" , ", ", ")
  generated = generated.replace(" : ", ": ")
  generated = generated.replace(" ? ", "? ")
  generated = generated.replace("did n't", "didn't")
  generated = generated.replace("should n't", "shouldn't")
  generated = generated.replace("do n't", "don't")
  generated = generated.replace("do esn't", "doesn't")
  generated = generated.replace("could n't", "couldn't")
  generated = generated.replace("gon na", "gonna")
  generated = generated.replace("got ta", "gotta")
  generated = generated.replace("was n't", "wasn't")

  print(generated)
  return generated

In [25]:
def save_metrics(rnn_size, batch_size, seq_length, num_epochs, learning_rate, sequences_step, history, generated, md):
  # Stream the metrics to a file in JSON format.
  json_log = open(save_dir + "/" +'log' + '_R' + str(rnn_size) + '_B' + str(batch_size) + '_L' + str(learning_rate) + '.json', mode='wt', buffering=1)
  json_log.write(
          json.dumps({'rnn_size': rnn_size, 
                      'batch_size': batch_size,
                      'seq_length': seq_length, 
                      'num_epochs': num_epochs, 
                      'learning_rate': learning_rate, 
                      'sequences_step': sequences_step, 

                      'history': history.history,

                      'predicted_text': generated
          })
  )
  json_log.close()

  # Open the file
  with open(save_dir + "/" +'model_summary' + '_R' + str(rnn_size) + '_B' + str(batch_size) + '_L' + str(learning_rate) + '.txt', mode='w') as fh:
      # Pass the file handle in as a lambda function to make it callable
      model.summary(line_length=80,print_fn=lambda x: fh.write(x + '\n')) 

Combinaciones

In [28]:
#Hiper-parametros
rnn_size = [256, 512, 1024] # size of RNN
batch_size = [32, 64] # minibatch size

seq_length = seq_length # sequence length
num_epochs = 30 # number of epochs
learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [29]:
for rnn in rnn_size:
   for batch in batch_size:
    md = bidirectional_lstm_model(seq_length, vocab_size, rnn, batch, learning_rate)
    print(md.summary())
    print('RNN Size= ' + str(rnn), ' Batch Size= ' + str(batch), ' Learning Rate= ' + str(learning_rate))
    history = fit_model(num_epochs, batch)
    #load_vocabulary()
    model = load_the_model()
    generated = ''
    sentence = []
    sentence, generated = initiate_sentences(sentence, generated)
    sentence, generated = generate_text(sentence, generated)
    generated = post_process(generated)
    save_metrics(rnn, batch, seq_length, num_epochs, learning_rate, sequences_step, history, generated, md) 

Build LSTM model.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 512)               9965568   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4609)              2364417   
_________________________________________________________________
lambda_2 (Lambda)            (None, 4609)              0         
_________________________________________________________________
activation_2 (Activation)    (None, 4609)              0         
Total params: 12,329,985
Trainable params: 12,329,985
Non-trainable params: 0
_________________________________________________________________
None
RNN Size= 256  Batch Size= 32  Learning Rate= 0.001
Epoch 1/30
1181/1181 [===========